In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('data/test.csv')
df.head()

,one,two,three,four,five
0,1.331587,0.715279,-1.545400,bar,NaN
1,NaN,NaN,1.000000,NaN,NaN
2,-0.008384,0.621336,-0.720086,bar,NaN
3,1.000000,0.500000,0.500000,NaN,NaN
4,0.265512,0.108549,0.004291,bar,yes


In [4]:
df_res = df.copy()
n_col = df_res.shape[0] * 0.5
df_res = df_res.dropna(how=any, thresh=n_col, axis=1)
n_row = df_res.shape[1]
df_res = df_res.dropna(how=any, thresh=n_row-2, axis=0)
values = {
    'one': df_res['one'].mean(),
    'two': df_res['two'].mean(),
    'four': df_res['four'].mode()[0]
}
df_res = df_res.fillna(values)
print(df_res)

        one       two     three  four
0  1.331587  0.715279 -1.545400   bar
2 -0.008384  0.621336 -0.720086   bar
3  1.000000  0.500000  0.500000   bar
4  0.265512  0.108549  0.004291   bar
5 -0.174600  0.433026  1.203037   bar
6  0.241508  0.567744  1.200000  hist
7 -0.965066  1.028274  0.228630   bar


In [5]:
diabetes_df = pd.read_csv('data/diabetes_data.csv')
diabetes_df.tail()
#print(diabetes_df.info())

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Gender
773,6,103,72,32,190,37.7,0.324,55,0,Female
774,1,71,48,18,76,20.4,0.323,22,0,Female
775,0,117,0,0,0,33.8,0.932,44,0,Female
776,4,154,72,29,126,31.3,0.338,37,0,Female
777,5,147,78,0,0,33.7,0.218,65,0,Female


In [6]:
work_df = diabetes_df.copy()
colomns_list = list(work_df.columns)
work_df = work_df.drop_duplicates(subset=colomns_list)
work_df.shape[0]

768

In [7]:
col_for_drop = []
for col in colomns_list:
    top_freq = work_df[col].value_counts(normalize=True).max()
    uniq_freq = work_df[col].nunique() / work_df[col].count()
    if top_freq > 0.99:
        col_for_drop.append(col)
    if uniq_freq >0.99:
        col_for_drop.append(col)
print(col_for_drop)
work_df = work_df.drop(col_for_drop, axis=1)

['Gender']


In [8]:
import numpy as np

zero_list = ['Glucose', 'BloodPressure', 'SkinThickness', 'Insulin', 'BMI']
null_df = work_df.copy()

for col in zero_list:
    null_df[col] = null_df[col].apply(lambda x: np.NaN if x==0 else x)

print(null_df['Insulin'].isnull().mean())

0.4869791666666667


In [9]:
drop_df = null_df.copy()
thresh = drop_df.shape[0] * 0.7
drop_df = drop_df.dropna(how='any', thresh=thresh, axis=1)
drop_df.shape[1]

8

In [10]:
n_col = drop_df.shape[1]
drop_df = drop_df.dropna(how='any', thresh=n_col-2, axis=0)
drop_df.shape[0]

761

In [11]:
drop_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 761 entries, 0 to 767
Data columns (total 8 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   Pregnancies               761 non-null    int64  
 1   Glucose                   756 non-null    float64
 2   BloodPressure             733 non-null    float64
 3   SkinThickness             541 non-null    float64
 4   BMI                       757 non-null    float64
 5   DiabetesPedigreeFunction  761 non-null    float64
 6   Age                       761 non-null    int64  
 7   Outcome                   761 non-null    int64  
dtypes: float64(5), int64(3)
memory usage: 53.5 KB


In [12]:
fillna_df = drop_df.copy()

"""
    values = dict()
    for col in fillna_df.columns:
        values[col] = f'fillna_df["{col}"].median()' # как ключу присвоить не строку, а выполнение
"""
values = {
    'Glucose': fillna_df['Glucose'].median(),
    'BloodPressure': fillna_df['BloodPressure'].median(),
    'SkinThickness': fillna_df['SkinThickness'].median(),
    'BMI': fillna_df['BMI'].median()
}
fillna_df = fillna_df.fillna(values)
print(fillna_df['SkinThickness'].mean())

29.109067017082786


In [13]:
def outliers_iqr(data, feature):
    x = data[feature]
    quartile_1, quartile_3 = x.quantile(0.25), x.quantile(0.75),
    iqr = quartile_3 - quartile_1
    lower_bound = quartile_1 - (iqr * 1.5)
    upper_bound = quartile_3 + (iqr * 1.5)
    outliers = data[(x<lower_bound) | (x > upper_bound)]
    cleaned = data[(x>lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_iqr(fillna_df, 'SkinThickness')
print(f'Число выбросов по методу Тьюки: {outliers.shape[0]}')

Число выбросов по методу Тьюки: 87


In [14]:
def outliers_z_score(data, feature, log_scale=False):
    if log_scale:
        x = np.log(data[feature])
    else:
        x = data[feature]
    mu = x.mean()
    sigma = x.std()
    lower_bound = mu - 3 * sigma
    upper_bound = mu + 3 * sigma
    outliers = data[(x < lower_bound) | (x > upper_bound)]
    cleaned = data[(x > lower_bound) & (x < upper_bound)]
    return outliers, cleaned

outliers, cleaned = outliers_z_score(fillna_df, 'SkinThickness')
print(f'Число выбросов по методу z-отклонения: {outliers.shape[0]}')

Число выбросов по методу z-отклонения: 4


In [15]:
outliers, cleaned = outliers_iqr(fillna_df, 'DiabetesPedigreeFunction')
outliers_z, cleaned_z = outliers_z_score(fillna_df, 'DiabetesPedigreeFunction', log_scale=True)
print(outliers.shape[0] - outliers_z.shape[0])

29
